In [3]:
import pandas as pd
import requests
import time

instrumentation_file = "./SWRC_DAP_Instrumentation_-6480797226837888374.csv"

In [4]:
instrumentation_df = pd.read_csv(instrumentation_file)
instrumentation_df.columns

Index(['OBJECTID', 'id', 'effectiveDate', 'instrument_ID', 'name', 'siteType',
       'Watershed', 'siteTypeStr', 'structure', 'dataLoc',
       'Type (Digital/Analog/Both)', 'Analog_Years', 'Purpose',
       'Digital_Years', 'State', 'County', 'Easting_DB', 'Norting_DB',
       'Elevation_DB', 'latitude', 'longitude', 'Area_acres', 'In_DB',
       'Published', 'Notes', 'Notes 2', 'x', 'y'],
      dtype='object')

In [5]:
# Let's get a look at the uniquie instrument type names.
siteTypes = pd.Series(instrumentation_df.siteTypeStr.unique())
siteTypes

0         Raingage
1            Flume
2             Weir
3             Tank
4              Met
5    Soil Moisture
6             Flux
7             Well
dtype: object

In [6]:
wgew_raingage_df = instrumentation_df.query("siteTypeStr == 'Raingage' and Watershed == 'WGEW'")

In [7]:
# We have all of the WGEW raingages.
# Iterate through each and download the precip data for each raingage.
# The url to the WGEW DAP site: https://www.tucson.ars.ag.gov/dap/
# The url to the WGEW DAP 1953-1999 raingage event page: https://www.tucson.ars.ag.gov/dap/event.asp
# Example url for POST querying a guage's event precip data: https://www.tucson.ars.ag.gov/dap/eventh.asp
# Example form 1953-1999 data: hiddenStartYear=1953&hiddenEndYear=1999&Watershed=63&gages=1-140&StartMonth=1&StartDay=1&StartYear=1953&EndMonth=12&EndDay=31&EndYear=1999&all=ON&type=summary&format=text&sortby=sortby_gage&units=inches&MinDepth=&MaxDepth=&MinDuration=&MaxDuration=&submit=Submit
# Example form 2000-2024 data: hiddenStartYear=2000&hiddenEndYear=2024&Watershed=63&gages=1-500&StartMonth=1&StartDay=1&StartYear=2000&EndMonth=12&EndDay=31&EndYear=2024&all=ON&type=summary&format=text&sortby=sortby_gage&units=inches&MinDepth=&MaxDepth=&MinDuration=&MaxDuration=&submit=Submit
# analog: /dap/eventh.asp, header: Gage,Date,Time,Duration,Depth,Time_Est,Rainfall_Rate,Rainfall_Est
# digital: /dap/digital/eventh.asp, header: Gage,Date,Time,Duration,Depth,,Rainfall_Rate

current_year = time.strftime("%Y")
time_ranges = {
    "analog": {"start_year": "1953", "end_year": "1999"},
    "digital": {"start_year": "2000", "end_year": f"{current_year}"},
}

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:132.0) Gecko/20100101 Firefox/132.0',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
    'Accept-Language': 'en-US,en;q=0.5',
    'Accept-Encoding': 'gzip, deflate, br, zstd',
    'Content-Type': 'application/x-www-form-urlencoded',
    'Content-Length': '215',
    'Origin': 'https://www.tucson.ars.ag.gov',
    'Connection': 'keep-alive',
    'Referer': 'https://www.tucson.ars.ag.gov/dap/digital/event.asp',
    'Cookie': 'ASPSESSIONIDSCFDDCDB=EOLLGDGDENMLKPANMNAKGEML',
    'Upgrade-Insecure-Requests': '1',
    'Sec-Fetch-Dest': 'document',
    'Sec-Fetch-Mode': 'navigate',
    'Sec-Fetch-Site': 'same-origin',
    'Sec-Fetch-User': '?1',
    'DNT': '1',
    'Sec-GPC': '1',
    'Priority': 'u=0, i'
}

payload = {
    "hiddenStartYear": "1953",
    "hiddenEndYear": "1999",
    "Watershed": "63",
    "gages": "1",
    "StartMonth": "1",
    "StartDay": "1",
    "StartYear": "1953",
    "EndMonth": "12",
    "EndDay": "31",
    "EndYear": "1999",
    "all": "ON",
    "type": "breakpoint",
    "format": "text",
    "sortby": "sortby_gage",
    "units": "inches",
    "MinDepth": "",
    "MaxDepth": "",
    "MinDuration": "",
    "MaxDuration": "",
    "submit": "Submit",
}

analog_url = "https://www.tucson.ars.ag.gov/dap/eventh.asp"
digital_url = "https://www.tucson.ars.ag.gov/dap/digital/eventh.asp"

# session = requests.Session()
# resp = session.post(url, headers=headers, data=payload)
# for line in resp.text.split("\n"):
#     print(line)